<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02-intermediate/Convolutinal_Neural_Network_using_KerasAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()
tfe = tf.contrib.eager
tfk = tf.keras

In [134]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 32
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tfk.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [135]:
x_train_ = tf.transpose(tf.convert_to_tensor(x_train, dtype=tf.float32), 
                        [0, 3, 1, 2])
y_train_ = tf.reshape(tf.one_hot(y_train, 10), (-1, 10))


print(x_train_.shape)
print(y_train_.shape)

(50000, 3, 32, 32)
(50000, 10)


In [136]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

train_dataset = train_dataset.repeat()
print(train_dataset)

<RepeatDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [137]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.div(tf.cast(
                          tf.transpose(x, [0, 3, 1, 2]), tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
test_dataset = test_dataset.repeat()
print(test_dataset)

<RepeatDataset shapes: ((?, 3, 32, 32), (?, 10)), types: (tf.float32, tf.float32)>


In [0]:
class Cifar10Model(tfk.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tfk.Sequential([
            tfk.layers.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])

        self.conv_block2 = tfk.Sequential([
            tfk.layers.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])
        
        self.conv_block3 = tfk.Sequential([
            tfk.layers.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_initializer=tf.initializers.variance_scaling,
                kernel_regularizer=tfk.regularizers.l2(l=0.001),
                data_format="channels_first"
            ),
            tfk.layers.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
                data_format="channels_first"
            ),
            tfk.layers.BatchNormalization(axis=1),
        ])
        
        self.flatten = tfk.layers.Flatten()
        self.fc1 = tfk.layers.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_initializer=tf.initializers.variance_scaling,
            kernel_regularizer=tfk.regularizers.l2(l=0.001))
        self.dropout = tfk.layers.Dropout(0.8)
        self.fc2 = tfk.layers.Dense(10)
        self.softmax = tfk.layers.Softmax()

    def call(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

In [140]:
model = Cifar10Model()
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
y_init = model(x_train_[:100])
y_init.shape

TensorShape([Dimension(100), Dimension(10)])

In [141]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_31 (Sequential)   multiple                  640       
_________________________________________________________________
sequential_32 (Sequential)   multiple                  3280      
_________________________________________________________________
sequential_33 (Sequential)   multiple                  12960     
_________________________________________________________________
flatten_14 (Flatten)         multiple                  0         
_________________________________________________________________
dense_24 (Dense)             multiple                  131328    
_________________________________________________________________
dropout_10 (Dropout)         multiple                  0         
_________________________________________________________________
dense_25 (Dense)             multiple                  2570      
__________

In [142]:
steps_per_epoch = int(x_train.shape[0]/batch_size)

model.fit(
    train_dataset,
    batch_size=batch_size,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
    shuffle=True,
    workers=5
)

Epoch 1/10
1562/1562 [==============================] - 94s 60ms/step - loss: 2.0714 - acc: 0.3507
Epoch 2/10
1562/1562 [==============================] - 93s 59ms/step - loss: 1.6550 - acc: 0.4786
Epoch 3/10
1562/1562 [==============================] - 99s 64ms/step - loss: 1.4775 - acc: 0.5425
Epoch 4/10
1562/1562 [==============================] - 98s 63ms/step - loss: 1.3834 - acc: 0.5780
Epoch 5/10
1562/1562 [==============================] - 94s 60ms/step - loss: 1.3222 - acc: 0.6098
Epoch 6/10
1562/1562 [==============================] - 94s 60ms/step - loss: 1.2834 - acc: 0.6273
Epoch 7/10
1562/1562 [==============================] - 95s 61ms/step - loss: 1.2596 - acc: 0.6392
Epoch 8/10
1562/1562 [==============================] - 95s 61ms/step - loss: 1.2430 - acc: 0.6477
Epoch 9/10
1562/1562 [==============================] - 92s 59ms/step - loss: 1.2210 - acc: 0.6587
Epoch 10/10
1562/1562 [==============================] - 95s 61ms/step - loss: 1.2108 - acc: 0.6642


In [144]:
train_loss, train_acc = model.evaluate(train_dataset, 
                                       steps=int(y_train.shape[0]/batch_size))
print("train_acc: {:0.3f}".format(train_acc))

test_loss, test_acc = model.evaluate(test_dataset, 
                                     steps=int(y_test.shape[0]/1000))
print("test_acc: {:0.3f}".format(test_acc))

1562/1562 [==============================] - 36s 23ms/step
train_acc: 0.720
10/10 [==============================] - 1s 136ms/step
test_acc: 0.688
